# Tratamento de dados

In [1]:
import os
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ftrav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ftrav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def count_words(text):
    words = text.split()
    return len(words)

In [4]:
def cut_voters(text):
    regex = re.compile('(?P<voters>.+) Votaram por essa decisão')
    try:
        text_cut = regex.search(str(text)).group('voters')
    except:
        text_cut = text
    return text_cut

In [5]:
def extract_meeting_number(sentence):
    match = re.search(r'\d+', sentence)
    if match:
        return match.group()
    else:
        return None

In [6]:
def preprocess_text(text):

    # Remove newline characters
    text = re.sub(r'\n', ' ', text)

    # Remove tab characters
    text = re.sub(r'\t', ' ', text)

    # Remove Unicode characters
    # text = re.sub(r'[^\x00-\x7F]+', '', text)

    # Convert text to lowercase
    text = text.lower()

    # Tokenize text into individual words
    words = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('portuguese'))
    stop_words.discard('sem')
    stop_words.discard('com')
    stop_words.discard('até')
    stop_words.discard('por')
    words = [word for word in words if word not in stop_words]

    # Join the words back into a single string
    processed_text = ' '.join(words)

    return processed_text

### Statements

In [7]:
df_statements = pd.read_csv('comunicados.csv')

In [8]:
df_statements.tail()

,Conteudo,Titulo,dataPublicacao,mostrarCabecalho,mostrarRodape,ultimaAtualizacao
204,<div> <p>O Copom decidiu manter a meta da taxa...,Nota à Imprensa - 50ª. Reunião,2000-08-23T03:00:00Z,False,False,2016-10-18T20:01:17Z
205,<div> <p>O Copom decidiu alterar a meta da tax...,Nota à Imprensa - 49ª. Reunião,2000-07-19T03:00:00Z,False,False,2016-10-18T20:01:17Z
206,<div> <p>Os resultados recentes da inflação mu...,Nota à Imprensa - 48ª. Reunião,2000-06-20T03:00:00Z,False,False,2016-10-18T20:01:17Z
207,<div> <p>O Copom analisando os eventos recente...,Nota à Imprensa - 47ª. Reunião,2000-05-24T03:00:00Z,False,False,2016-10-18T20:01:17Z
208,<div> <p>Em que pese as recentes trajetórias f...,Nota à Imprensa - 46ª. Reunião,2000-04-19T03:00:00Z,False,False,2016-10-18T20:01:16Z


In [9]:
df_statements.loc[50:,'Titulo']

50     Copom reduz a taxa Selic para 13,00% ao ano
51     Copom reduz a taxa Selic para 13,75% ao ano
52     Copom reduz a taxa Selic para 14,00% ao ano
53                 Nota à Imprensa - 201ª. Reunião
54                 Nota à Imprensa - 200ª. Reunião
                          ...                     
204                 Nota à Imprensa - 50ª. Reunião
205                 Nota à Imprensa - 49ª. Reunião
206                 Nota à Imprensa - 48ª. Reunião
207                 Nota à Imprensa - 47ª. Reunião
208                 Nota à Imprensa - 46ª. Reunião
Name: Titulo, Length: 159, dtype: object

In [10]:
df_statements['meeting_number'] = df_statements['Titulo'].apply(extract_meeting_number).astype(int)

In [11]:
df_statements.head()

,Conteudo,Titulo,dataPublicacao,mostrarCabecalho,mostrarRodape,ultimaAtualizacao,meeting_number
0,"<p style=""text-align&#58;justify;"">O ambiente ...","Copom mantém a taxa Selic em 13,75% a.a.",2023-05-03T21:46:13Z,False,False,2023-05-03T21:46:14Z,13
1,"<p style=""text-align&#58;justify;"">Desde a reu...","Copom mantém a taxa Selic em 13,75% a.a.",2023-03-22T21:31:40Z,False,False,2023-03-22T21:31:41Z,13
2,"<p style=""text-align&#58;justify;"">Em sua 252ª...","Copom mantém a taxa Selic em 13,75% a.a.",2023-02-01T21:34:30Z,False,False,2023-02-01T21:34:30Z,13
3,"<p style=""text-align&#58;justify;"">Em sua 251ª...","Copom mantém a taxa Selic em 13,75% a.a.",2022-12-07T21:31:04Z,False,False,2022-12-07T21:31:04Z,13
4,"<p style=""text-align&#58;justify;"">Em sua 250ª...","Copom mantém a taxa Selic em 13,75% a.a.",2022-10-26T21:35:29Z,False,False,2022-10-26T21:35:30Z,13


In [12]:
df_statements.loc[50:,'meeting_number']

50      13
51      13
52      14
53     201
54     200
      ... 
204     50
205     49
206     48
207     47
208     46
Name: meeting_number, Length: 159, dtype: int32

In [13]:
for i in range(53, 0, -1):
    number = df_statements.iloc[i, 6]
    df_statements.iloc[i-1, 6] = number + 1

In [14]:
df_statements.loc[:,'meeting_number']

0      254
1      253
2      252
3      251
4      250
      ... 
204     50
205     49
206     48
207     47
208     46
Name: meeting_number, Length: 209, dtype: int32

In [15]:
df_statements['type'] = 'statement'
df_statements["date"] = df_statements["dataPublicacao"].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S.%f').dt.strftime('%Y/%m/%d')
df_statements.drop(['dataPublicacao', 'Titulo', 'ultimaAtualizacao', 'mostrarCabecalho', 'mostrarRodape'], axis=1, inplace=True)

In [16]:
df_statements.rename(columns={'Conteudo': 'text_raw'}, inplace=True)

In [17]:
# df_statements.sort_values('Date', inplace=True)
# df_statements.set_index('Date', inplace=True)
df_statements = df_statements.sort_values('meeting_number').copy()

In [18]:
df_statements['text_raw'] = df_statements['text_raw'].apply(lambda x: BeautifulSoup(x, features="html.parser").get_text())

In [19]:
df_statements.head()

,text_raw,meeting_number,type,date
208,Em que pese as recentes trajetórias favorávei...,46,statement,2000/04/19
207,O Copom analisando os eventos recentes e pros...,47,statement,2000/05/24
206,Os resultados recentes da inflação mudaram si...,48,statement,2000/06/20
205,O Copom decidiu alterar a meta da taxa Selic ...,49,statement,2000/07/19
204,O Copom decidiu manter a meta da taxa Selic e...,50,statement,2000/08/23


In [20]:
# df_statements['Text_raw'][208]

In [21]:
df_statements['text'] = df_statements['text_raw'].apply(cut_voters).apply(preprocess_text)

In [22]:
# df_statements['Text'][208]

In [23]:
df_statements['num_words'] = df_statements['text'].apply(count_words).astype(int)
df_statements['num_words_raw'] = df_statements['text_raw'].apply(count_words).astype(int)

In [24]:
df_statements.head(5)

,text_raw,meeting_number,type,date,text,num_words,num_words_raw
208,Em que pese as recentes trajetórias favorávei...,46,statement,2000/04/19,pese recentes trajetórias favoráveis taxas inf...,51,54
207,O Copom analisando os eventos recentes e pros...,47,statement,2000/05/24,copom analisando eventos recentes prospectivos...,15,22
206,Os resultados recentes da inflação mudaram si...,48,statement,2000/06/20,resultados recentes inflação mudaram significa...,29,36
205,O Copom decidiu alterar a meta da taxa Selic ...,49,statement,2000/07/19,"copom decidiu alterar meta taxa selic 17,0 % 1...",23,37
204,O Copom decidiu manter a meta da taxa Selic e...,50,statement,2000/08/23,"copom decidiu manter meta taxa selic 16,50 % e...",17,23


In [25]:
df_statements.shape

(209, 7)

In [26]:
# df_statements.drop(['text_raw', 'num_words_raw'], axis=1, inplace=True)

In [27]:
df_statements

,text_raw,meeting_number,type,date,text,num_words,num_words_raw
208,Em que pese as recentes trajetórias favorávei...,46,statement,2000/04/19,pese recentes trajetórias favoráveis taxas inf...,51,54
207,O Copom analisando os eventos recentes e pros...,47,statement,2000/05/24,copom analisando eventos recentes prospectivos...,15,22
206,Os resultados recentes da inflação mudaram si...,48,statement,2000/06/20,resultados recentes inflação mudaram significa...,29,36
205,O Copom decidiu alterar a meta da taxa Selic ...,49,statement,2000/07/19,"copom decidiu alterar meta taxa selic 17,0 % 1...",23,37
204,O Copom decidiu manter a meta da taxa Selic e...,50,statement,2000/08/23,"copom decidiu manter meta taxa selic 16,50 % e...",17,23
...,...,...,...,...,...,...,...
4,"Em sua 250ª reunião, o Comitê de Política Mone...",250,statement,2022/10/26,"250ª reunião , comitê política monetária ( cop...",532,814
3,"Em sua 251ª reunião, o Comitê de Política Mone...",251,statement,2022/12/07,"251ª reunião , comitê política monetária ( cop...",550,832
2,"Em sua 252ª reunião, o Comitê de Política Mone...",252,statement,2023/02/01,"252ª reunião , comitê política monetária ( cop...",520,896
1,Desde a reunião anterior do Comitê de Política...,253,statement,2023/03/22,desde reunião anterior comitê política monetár...,606,845


### Minutes

In [28]:
df_minutes = pd.read_csv('atas.csv')

In [29]:
df_minutes.tail(1)

,BoxDestaque,DataReferencia,Edicao,EsconderCapa,EsconderDataReferencia,Grafico,Id,ImagemCapa,Introducao,LinkPagina,OutrasInformacoes,Titulo,Url,Volume,data
233,NaN,1998-01-28T02:00:00Z,NaN,True,NaN,NaN,191,/content/publicacoes/PublishingImages/Capas/at...,NaN,/publicacoes/atascopom/cronologicos,"\n\n \n<h3>Sumário</h3><a href=""#_Toc37333813...",21ª Reunião,NaN,NaN,1998-05-22T03:00:00Z


In [30]:
df_minutes.loc[:,'Titulo']

0                      254ª Reunião - 2-3 maio, 2023
1                   253ª Reunião - 21-22 março, 2023
2      252ª Reunião - 31 janeiro - 1 fevereiro, 2023
3                  251ª Reunião - 6-7 dezembro, 2022
4                 250ª Reunião - 25-26 outubro, 2022
                           ...                      
229                                      25ª Reunião
230                                      24ª Reunião
231                                      23ª Reunião
232                                      22ª Reunião
233                                      21ª Reunião
Name: Titulo, Length: 234, dtype: object

In [31]:
# df_minutes.iloc[150:164,[1,14]]

In [32]:
df_minutes['meeting_number'] = df_minutes['Titulo'].apply(extract_meeting_number).astype(int)

In [33]:
df_minutes.tail()

,BoxDestaque,DataReferencia,Edicao,EsconderCapa,EsconderDataReferencia,Grafico,Id,ImagemCapa,Introducao,LinkPagina,OutrasInformacoes,Titulo,Url,Volume,data,meeting_number
229,NaN,1998-06-01T03:00:00Z,NaN,True,NaN,NaN,95,/content/publicacoes/PublishingImages/Capas/at...,NaN,/publicacoes/atascopom/cronologicos,\n\n \n <p>&nbsp;</p>\n\n <p><strong>Su...,25ª Reunião,NaN,NaN,1998-10-09T03:00:00Z,25
230,NaN,1998-05-01T03:00:00Z,NaN,True,NaN,NaN,96,/content/publicacoes/PublishingImages/Capas/at...,NaN,/publicacoes/atascopom/cronologicos,"\n \n <div class=""h3"">Sum&aacute;rio</div>...",24ª Reunião,NaN,NaN,1998-08-04T03:00:00Z,24
231,NaN,1998-04-01T03:00:00Z,NaN,True,NaN,NaN,97,/content/publicacoes/PublishingImages/Capas/at...,NaN,/publicacoes/atascopom/cronologicos,"\n \n <div class=""h3"">Sum&aacute;rio</div>...",23ª Reunião,NaN,NaN,1998-07-31T03:00:00Z,23
232,NaN,1998-03-01T03:00:00Z,NaN,True,NaN,NaN,98,/content/publicacoes/PublishingImages/Capas/at...,NaN,/publicacoes/atascopom/cronologicos,"\n \n <div class=""h3"">Sum&aacute;rio</div>...",22ª Reunião,NaN,NaN,1998-06-26T03:00:00Z,22
233,NaN,1998-01-28T02:00:00Z,NaN,True,NaN,NaN,191,/content/publicacoes/PublishingImages/Capas/at...,NaN,/publicacoes/atascopom/cronologicos,"\n\n \n<h3>Sumário</h3><a href=""#_Toc37333813...",21ª Reunião,NaN,NaN,1998-05-22T03:00:00Z,21


In [34]:
df_minutes['type'] = 'minutes'
# df_minutes["Minutes_Date"] = df_minutes["data"].apply(pd.to_datetime).dt.strftime('%Y/%m/%d')
df_minutes["date"] = df_minutes["DataReferencia"].apply(pd.to_datetime).dt.strftime('%Y/%m/%d')

In [35]:
len(df_minutes)

234

In [36]:
df_minutes.drop(['data', 'DataReferencia', 'BoxDestaque', 'Edicao', 'EsconderCapa', 
                 'EsconderDataReferencia', 'Grafico', 'Id', 
                 'ImagemCapa', 'LinkPagina', 'Url',
                 'Titulo', 'Volume', 'Introducao'], axis=1, inplace=True)

In [37]:
df_minutes.rename(columns={'OutrasInformacoes': 'text_raw'}, inplace=True)

In [38]:
# df_minutes.sort_values('Date', inplace=True)
# df_minutes.set_index('Date', inplace=True)
df_minutes = df_minutes.sort_values('meeting_number').copy()

In [39]:
df_minutes

,text_raw,meeting_number,type,date
233,"\n\n \n<h3>Sumário</h3><a href=""#_Toc37333813...",21,minutes,1998/01/28
232,"\n \n <div class=""h3"">Sum&aacute;rio</div>...",22,minutes,1998/03/01
231,"\n \n <div class=""h3"">Sum&aacute;rio</div>...",23,minutes,1998/04/01
230,"\n \n <div class=""h3"">Sum&aacute;rio</div>...",24,minutes,1998/05/01
229,\n\n \n <p>&nbsp;</p>\n\n <p><strong>Su...,25,minutes,1998/06/01
...,...,...,...,...
4,"<div id=""atacompleta""><div id=""ataconteudo""><h...",250,minutes,2022/10/26
3,"<div id=""atacompleta""><div id=""ataconteudo""><h...",251,minutes,2022/12/07
2,"<div id=""atacompleta""><div id=""ataconteudo""><h...",252,minutes,2023/02/01
1,"<div id=""atacompleta""><div id=""ataconteudo""><h...",253,minutes,2023/03/22


In [40]:
df_minutes = df_minutes[df_minutes['text_raw'].notna()]

In [41]:
df_minutes['text_raw'] = df_minutes['text_raw'].apply(lambda x: BeautifulSoup(x, features="html.parser").get_text())

C:\Users\ftrav\AppData\Local\Temp\ipykernel_11168\586401199.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minutes['text_raw'] = df_minutes['text_raw'].apply(lambda x: BeautifulSoup(x, features="html.parser").get_text())


In [42]:
# df_minutes['Text_raw'][200]

In [43]:
df_minutes['text'] = df_minutes['text_raw'].apply(cut_voters).apply(preprocess_text)

C:\Users\ftrav\AppData\Local\Temp\ipykernel_11168\1530529726.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minutes['text'] = df_minutes['text_raw'].apply(cut_voters).apply(preprocess_text)


In [44]:
# df_minutes['Text'][200]

In [45]:
df_minutes['num_words'] = df_minutes['text'].apply(count_words).astype(int)
df_minutes['num_words_raw'] = df_minutes['text_raw'].apply(count_words).astype(int)

C:\Users\ftrav\AppData\Local\Temp\ipykernel_11168\1668320117.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minutes['num_words'] = df_minutes['text'].apply(count_words).astype(int)
C:\Users\ftrav\AppData\Local\Temp\ipykernel_11168\1668320117.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minutes['num_words_raw'] = df_minutes['text_raw'].apply(count_words).astype(int)


In [46]:
df_minutes.shape

(202, 7)

In [47]:
# df_minutes.iloc[200:,[1,14]]

In [48]:
# df_minutes.drop(['text_raw', 'num_words_raw'], axis=1, inplace=True)

In [49]:
df_minutes.head()

,text_raw,meeting_number,type,date,text,num_words,num_words_raw
233,\nSumárioPreços e Nível de Atividade\nAgregado...,21,minutes,1998/01/28,sumáriopreços nível atividade agregados monetá...,3926,5066
232,\nSumárioPreços e Nível de Atividade\nAgregado...,22,minutes,1998/03/01,sumáriopreços nível atividade agregados monetá...,2961,3893
231,\nSumárioPreços e Nível de Atividade\nAgregado...,23,minutes,1998/04/01,sumáriopreços nível atividade agregados monetá...,3899,4945
230,\nSumárioPreços e Nível de Atividade\nAgregado...,24,minutes,1998/05/01,sumáriopreços nível atividade agregados monetá...,3367,4341
229,\n \nSumário\n \nPreços e Nível de Atividade\n...,25,minutes,1998/06/01,sumário preços nível atividade agregados monet...,5246,6626


### Decisions

In [50]:
df_decisions = pd.read_csv('decisions.csv')

In [51]:
df_decisions.loc[:,'NumeroReuniaoCopom']

0      254.0
1      253.0
2      252.0
3      251.0
4      250.0
       ...  
257      5.0
258      4.0
259      3.0
260      2.0
261      1.0
Name: NumeroReuniaoCopom, Length: 262, dtype: float64

In [52]:
df_decisions['NumeroReuniaoCopom'].value_counts()

34.0     4
33.0     3
48.0     2
35.0     2
45.0     2
        ..
165.0    1
164.0    1
163.0    1
162.0    1
1.0      1
Name: NumeroReuniaoCopom, Length: 254, dtype: int64

In [53]:
# df_decisions.iloc[150:164,[0, 1, 2]]

In [54]:
df_decisions.drop(['DataFimVigencia', 'DataInicioVigencia', 'TaxaSelicEfetivaAnualizada',
                 'TaxaSelicEfetivaVigencia', 'TaxaTban', 'UsoMetaSelic', 'ReuniaoExtraordinaria',
                 'Vies'], axis=1, inplace=True)

In [55]:
df_decisions["DataReuniaoCopom"] = df_decisions["DataReuniaoCopom"].apply(pd.to_datetime).dt.strftime('%Y/%m/%d')

In [56]:
df_decisions.rename(columns={'MetaSelic': 'selic', 
                             'NumeroReuniaoCopom': 'meeting_number',
                             'DataReuniaoCopom': 'date'
                            }, inplace=True)

In [57]:
df_decisions['meeting_number'] = df_decisions['meeting_number'].astype(int).copy()

In [58]:
df_decisions = df_decisions[df_decisions['date'].notna()]

In [59]:
# df_decisions = df_decisions.sort_values('Date').copy()
# df_decisions.set_index('Date', inplace=True)
df_decisions = df_decisions.sort_values('meeting_number').copy()

In [60]:
df_decisions['decision'] = np.nan
df_decisions['decision_txt'] = ''

In [61]:
# df_decisions = df_decisions[['Selic', 'meeting_number', 'Decision', 'Decision_txt', 'Date']]

In [62]:
df_decisions['meeting_number'].count()

254

In [63]:
for i in range(len(df_decisions)-1):
    df_decisions.iloc[i+1,3] = df_decisions.iloc[i+1,1] - df_decisions.iloc[i,1]

In [64]:
def decision_txt(num):
    if num == 0:
        return 'manter'
    elif num >= 0:
        return 'elevar'
    elif num <= 0:
        return 'cortar'
    else:
        return ''

In [65]:
for i in range(len(df_decisions)-1):
    df_decisions.iloc[i+1,4] = decision_txt(df_decisions.iloc[i+1,3])

In [66]:
df_decisions.head()

,date,selic,meeting_number,decision,decision_txt
261,1996/06/26,1.90,1,NaN,
260,1996/07/30,1.90,2,0.00,manter
259,1996/08/21,1.88,3,-0.02,cortar
258,1996/09/23,1.82,4,-0.06,cortar
257,1996/10/23,1.78,5,-0.04,cortar


### Labels

In [67]:
df_decisions['label_hawk_dove'] = None
df_decisions['label_next_meet'] = None

In [68]:
for i in range(len(df_decisions)-1):
    decisao_atual = df_decisions.iloc[i, 3]
    decisao_seguite =  df_decisions.iloc[i+1, 3]
    decisao_seguite_txt = df_decisions.iloc[i+1, 4]    
        
    if decisao_seguite_txt == 'cortar':
        if decisao_seguite == decisao_atual:
            resultado = 'dovish'
        elif decisao_seguite > decisao_atual:
            resultado = 'dovish' # less dovish
        else:
            resultado = 'dovish' # more dovish
    elif decisao_seguite_txt == 'elevar':
        if decisao_seguite == decisao_atual:
            resultado = 'hawkish'
        elif decisao_seguite > decisao_atual:
            resultado = 'hawkish' # more hawkish
        else:
            resultado = 'hawkish' # less hawkish
    elif decisao_seguite_txt == 'manter':
        if decisao_atual < 0: 
            resultado = 'hawkish' # parou de cortar
        elif decisao_atual > 0:
            resultado = 'dovish' # parou de elevar
        else:
            resultado = 'neutral' # manteve
    
    df_decisions.iloc[i, 5] = resultado 

In [69]:
# df_decisions.head()

In [70]:
for i in range(len(df_decisions)-1):
    df_decisions.iloc[i, 6] = df_decisions.iloc[i+1, 4]

In [71]:
df_decisions

,date,selic,meeting_number,decision,decision_txt,label_hawk_dove,label_next_meet
261,1996/06/26,1.90,1,NaN,,neutral,manter
260,1996/07/30,1.90,2,0.00,manter,dovish,cortar
259,1996/08/21,1.88,3,-0.02,cortar,dovish,cortar
258,1996/09/23,1.82,4,-0.06,cortar,dovish,cortar
257,1996/10/23,1.78,5,-0.04,cortar,dovish,cortar
...,...,...,...,...,...,...,...
4,2022/10/26,13.75,250,0.00,manter,neutral,manter
3,2022/12/07,13.75,251,0.00,manter,neutral,manter
2,2023/02/01,13.75,252,0.00,manter,neutral,manter
1,2023/03/22,13.75,253,0.00,manter,neutral,manter


### Merge columns

In [72]:
# df_minutes.head()

In [73]:
# df_minutes.tail()

In [74]:
# df_statements.head()

In [75]:
# df_statements.tail()

In [76]:
df1 = pd.merge(df_statements, df_minutes, how="outer")

In [77]:
df1 = df1.sort_values('meeting_number').copy()

In [78]:
df1

,text_raw,meeting_number,type,date,text,num_words,num_words_raw
209,\nSumárioPreços e Nível de Atividade\nAgregado...,21,minutes,1998/01/28,sumáriopreços nível atividade agregados monetá...,3926,5066
210,\nSumárioPreços e Nível de Atividade\nAgregado...,22,minutes,1998/03/01,sumáriopreços nível atividade agregados monetá...,2961,3893
211,\nSumárioPreços e Nível de Atividade\nAgregado...,23,minutes,1998/04/01,sumáriopreços nível atividade agregados monetá...,3899,4945
212,\nSumárioPreços e Nível de Atividade\nAgregado...,24,minutes,1998/05/01,sumáriopreços nível atividade agregados monetá...,3367,4341
213,\n \nSumário\n \nPreços e Nível de Atividade\n...,25,minutes,1998/06/01,sumário preços nível atividade agregados monet...,5246,6626
...,...,...,...,...,...,...,...
408,A) Atualização da conjuntura econômica e do ce...,252,minutes,2023/02/01,) atualização conjuntura econômica cenário cop...,1937,3247
207,Desde a reunião anterior do Comitê de Política...,253,statement,2023/03/22,desde reunião anterior comitê política monetár...,606,845
409,A) Atualização da conjuntura econômica e do ce...,253,minutes,2023/03/22,) atualização conjuntura econômica cenário cop...,2013,3359
208,O ambiente externo se mantém adverso. Os episó...,254,statement,2023/05/03,ambiente externo mantém adverso . episódios en...,469,835


In [79]:
df_decisions

,date,selic,meeting_number,decision,decision_txt,label_hawk_dove,label_next_meet
261,1996/06/26,1.90,1,NaN,,neutral,manter
260,1996/07/30,1.90,2,0.00,manter,dovish,cortar
259,1996/08/21,1.88,3,-0.02,cortar,dovish,cortar
258,1996/09/23,1.82,4,-0.06,cortar,dovish,cortar
257,1996/10/23,1.78,5,-0.04,cortar,dovish,cortar
...,...,...,...,...,...,...,...
4,2022/10/26,13.75,250,0.00,manter,neutral,manter
3,2022/12/07,13.75,251,0.00,manter,neutral,manter
2,2023/02/01,13.75,252,0.00,manter,neutral,manter
1,2023/03/22,13.75,253,0.00,manter,neutral,manter


In [80]:
df2 = pd.merge(df1, df_decisions, how="outer", on=['meeting_number'])

In [81]:
df = df2.sort_values('meeting_number').copy()

In [82]:
df.head()

,text_raw,meeting_number,type,date_x,text,num_words,num_words_raw,date_y,selic,decision,decision_txt,label_hawk_dove,label_next_meet
411,NaN,1,NaN,NaN,NaN,NaN,NaN,1996/06/26,1.90,NaN,,neutral,manter
412,NaN,2,NaN,NaN,NaN,NaN,NaN,1996/07/30,1.90,0.00,manter,dovish,cortar
413,NaN,3,NaN,NaN,NaN,NaN,NaN,1996/08/21,1.88,-0.02,cortar,dovish,cortar
414,NaN,4,NaN,NaN,NaN,NaN,NaN,1996/09/23,1.82,-0.06,cortar,dovish,cortar
415,NaN,5,NaN,NaN,NaN,NaN,NaN,1996/10/23,1.78,-0.04,cortar,dovish,cortar


In [83]:
df = df[df['selic'].notna()]

In [84]:
df = df[df['text'].notna()]

In [85]:
df = df[['meeting_number', 'date_x', 'date_y', 'selic', 'decision', 'decision_txt', 
         'type', 'text', 'num_words', 'text_raw', 'num_words_raw', 'label_hawk_dove', 'label_next_meet']]

In [86]:
df

,meeting_number,date_x,date_y,selic,decision,decision_txt,type,text,num_words,text_raw,num_words_raw,label_hawk_dove,label_next_meet
0,21,1998/01/28,1998/01/28,34.50,-3.50,cortar,minutes,sumáriopreços nível atividade agregados monetá...,3926.0,\nSumárioPreços e Nível de Atividade\nAgregado...,5066.0,dovish,cortar
1,22,1998/03/01,1998/03/04,28.00,-6.50,cortar,minutes,sumáriopreços nível atividade agregados monetá...,2961.0,\nSumárioPreços e Nível de Atividade\nAgregado...,3893.0,dovish,cortar
2,23,1998/04/01,1998/04/15,23.25,-4.75,cortar,minutes,sumáriopreços nível atividade agregados monetá...,3899.0,\nSumárioPreços e Nível de Atividade\nAgregado...,4945.0,dovish,cortar
3,24,1998/05/01,1998/05/20,21.75,-1.50,cortar,minutes,sumáriopreços nível atividade agregados monetá...,3367.0,\nSumárioPreços e Nível de Atividade\nAgregado...,4341.0,dovish,cortar
4,25,1998/06/01,1998/06/24,21.00,-0.75,cortar,minutes,sumário preços nível atividade agregados monet...,5246.0,\n \nSumário\n \nPreços e Nível de Atividade\n...,6626.0,dovish,cortar
...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,252,2023/02/01,2023/02/01,13.75,0.00,manter,minutes,) atualização conjuntura econômica cenário cop...,1937.0,A) Atualização da conjuntura econômica e do ce...,3247.0,neutral,manter
407,253,2023/03/22,2023/03/22,13.75,0.00,manter,statement,desde reunião anterior comitê política monetár...,606.0,Desde a reunião anterior do Comitê de Política...,845.0,neutral,manter
408,253,2023/03/22,2023/03/22,13.75,0.00,manter,minutes,) atualização conjuntura econômica cenário cop...,2013.0,A) Atualização da conjuntura econômica e do ce...,3359.0,neutral,manter
410,254,2023/05/03,2023/05/03,13.75,0.00,manter,minutes,) atualização conjuntura econômica cenário cop...,1801.0,A) Atualização da conjuntura econômica e do ce...,3076.0,None,None


In [87]:
df.to_csv('df_copom_label_pt.csv', index=False)